In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

import LiwcTrie as lt
from Liwc import LiwcAnalyzer

from fbdp import FBDesktopParser

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alec\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Alec\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Read the data
pages = pd.read_csv('Data_Clean/pages.csv')
posts = pd.read_csv('Data_Clean/posts.csv')
posts.head()

In [8]:
#Clean the data
posts.drop('Unnamed: 0', inplace=True, axis=1)

In [9]:
print(posts.columns.values)

['article_host' 'article_name' 'article_subtitle' 'hashtags' 'img-label'
 'img_src' 'linked_profiles' 'links' 'text' 'timestamp' 'has_article'
 'text_length' 'num_hashtags' 'has_text' 'num_linked_profiles' 'num_links'
 'text_tokenized' 'num_tokens' 'text_tokenized_filtered'
 'text_tokenized_lemmatized' 'num_words' 'num_periods' 'percent_periods'
 'num_exclamations' 'percent_exclamations' 'num_questionms'
 'percent_questionms' 'num_equals' 'percent_equals' 'num_dollars'
 'percent_dollars' 'ttr' 'page_name' 'anti_vax' 'sentiment_neg'
 'sentiment_neu' 'sentiment_pos' 'sentiment_compound'
 'readability_smog_index' 'readability_gunning_fog'
 'readability_flesch_kincaid_grade' 'wc' 'sc' 'wps' 'dic' 'sixltr' 'funct'
 'pronoun' 'ppron' 'i' 'we' 'you' 'shehe' 'they' 'ipron' 'article' 'verbs'
 'auxvb' 'past' 'present' 'future' 'adverbs' 'prep' 'conj' 'negate'
 'quant' 'numbers' 'swear' 'social' 'family' 'friends' 'humans' 'affect'
 'posemo' 'negemo' 'anx' 'anger' 'sad' 'cogmech' 'insight' 'cause

In [12]:
#Describe the data
posts.describe()

,text_length,num_hashtags,num_linked_profiles,num_links,num_tokens,num_words,num_periods,percent_periods,num_exclamations,percent_exclamations,...,work,achiev,leisure,home,money,relig,death,assent,nonflu,filler
count,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,4.867419e+06,5.931405e+06,4.867419e+06,...,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06,5.931405e+06
mean,3.031692e+02,1.702055e-01,1.315634e+00,3.682886e-01,5.846567e+01,3.130572e+01,2.150639e+00,3.601236e-02,5.133455e-01,2.077935e-02,...,1.801275e-01,1.124353e-01,1.396795e-01,7.160004e-02,8.927398e-02,1.223936e-01,6.697968e-02,2.209767e-02,1.349612e-02,1.842762e-02
std,7.871881e+02,4.436411e-01,6.852855e-01,1.550264e+00,1.489931e+02,8.262925e+01,6.370215e+00,5.014587e-02,1.448722e+00,5.613781e-02,...,1.346815e-01,1.036299e-01,1.152809e-01,8.348828e-02,9.473943e-02,1.063576e-01,7.217305e-02,5.723709e-02,3.258250e-02,4.083505e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.130000e+02,0.000000e+00,1.000000e+00,0.000000e+00,2.200000e+01,1.200000e+01,1.000000e+00,2.877698e-02,0.000000e+00,0.000000e+00,...,2.000000e-01,1.132075e-01,1.500000e-01,6.273063e-02,8.333333e-02,1.283784e-01,6.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.900000e+02,0.000000e+00,2.000000e+00,1.000000e+00,5.500000e+01,3.000000e+01,2.000000e+00,5.000000e-02,1.000000e+00,1.612903e-02,...,2.666667e-01,1.666667e-01,2.068966e-01,1.093750e-01,1.346154e-01,1.851852e-01,1.063830e-01,2.962963e-02,1.600000e-02,2.222222e-02
max,4.024400e+04,2.200000e+01,2.800000e+01,1.180000e+02,6.323000e+03,5.152000e+03,2.320000e+02,1.000000e+00,6.000000e+01,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [14]:
posts.dtypes

article_host                  object
article_name                  object
article_subtitle              object
hashtags                      object
img-label                     object
img_src                       object
linked_profiles               object
links                         object
text                          object
timestamp                     object
has_article                     bool
text_length                    int64
num_hashtags                   int64
has_text                        bool
num_linked_profiles            int64
num_links                      int64
text_tokenized                object
num_tokens                     int64
text_tokenized_filtered       object
text_tokenized_lemmatized     object
num_words                      int64
num_periods                    int64
percent_periods              float64
num_exclamations               int64
percent_exclamations         float64
num_questionms                 int64
percent_questionms           float64
n

In [23]:
c = {
    'swear': 'float32',
    'tentat': 'float32',
    'posemo': 'float32',
    'assent': 'float32',
    'num_tokens': 'float32',
    'num_linked_profiles': 'float32',
    'certain': 'float32',
    'anti_vax': 'float32',
    'sentiment_compound': 'float32',
    'num_exclamations': 'float32',
    'prep': 'float32',
    'friends': 'float32',
    'present': 'float32',
    'img-label': 'float32',
    'social': 'float32',
    'page_name_adjusted': 'float32',
    'auxvb': 'float32',
    'insight': 'float32',
    'excl': 'float32',
    'percent_dollars': 'float32',
    'percent_questionms': 'float32',
    'sentiment_neu': 'float32',
    'negemo': 'float32',
    'article_subtitle': 'float32',
    'img_src': 'float32',
    'ingest': 'float32',
    'num_links': 'float32',
    'relig': 'float32',
    'feel': 'float32',
    'we': 'float32',
    'quant': 'float32',
    'text_tokenized_filtered': 'float32',
    'conj': 'float32',
    'leisure': 'float32',
    'sc': 'float32',
    'links': 'float32',
    'home': 'float32',
    'text_length': 'float32',
    'future': 'float32',
    'sixltr': 'float32',
    'num_hashtags': 'float32',
    'num_dollars': 'float32',
    'hashtags': 'float32',
    'hear': 'float32',
    'past': 'float32',
    'num_equals': 'float32',
    'anx': 'float32',
    'percent_equals': 'float32',
    'percept': 'float32',
    'i': 'float32',
    'see': 'float32',
    'timestamp': 'float32',
    'health': 'float32',
    'cogmech': 'float32',
    'text_tokenized': 'float32',
    'percent_exclamations': 'float32',
    'incl': 'float32',
    'humans': 'float32',
    'work': 'float32',
    'family': 'float32',
    'text_tokenized_lemmatized': 'float32',
    'num_words': 'float32',
    'numbers': 'float32',
    'percent_periods': 'float32',
    'you': 'float32',
    'death': 'float32',
    'readability_gunning_fog': 'float32',
    'motion': 'float32',
    'relativ': 'float32',
    'readability_smog_index': 'float32',
    'sentiment_neg': 'float32',
    'num_periods': 'float32',
    'theyipronarticleverbs': 'float32',
    'readability_flesch_kincaid_grade': 'float32',
    'linked_profiles': 'float32',
    'filler': 'float32',
    'space': 'float32',
    'ppron': 'float32',
    'has_article': 'float32',
    'sexual': 'float32',
    'shehe': 'float32',
    'money': 'float32',
    'dic': 'float32',
    'page_name': 'float32',
    'wps': 'float32',
    'nonflu': 'float32',
    'num_questionms': 'float32',
    'affect': 'float32',
    'text': 'float32',
    'has_text': 'float32',
    'article_name': 'float32',
    'funct': 'float32',
    'article_host': 'float32',
    'adverbs': 'float32',
    'pronoun': 'float32',
    'discrep': 'float32',
    'inhib': 'float32',
    'body': 'float32',
    'achiev': 'float32',
    'sad': 'float32',
    'cause': 'float32',
    'time': 'float32',
    'ttr': 'float32',
    'wc': 'float32',
    'negate': 'float32',
    'bio': 'float32',
    'anger': 'float32',
    'sentiment_pos': 'float32'
}

In [24]:
{x : 'float32' for x in c}

{'swear': 'float32',
 'tentat': 'float32',
 'posemo': 'float32',
 'assent': 'float32',
 'num_tokens': 'float32',
 'num_linked_profiles': 'float32',
 'certain': 'float32',
 'anti_vax': 'float32',
 'sentiment_compound': 'float32',
 'num_exclamations': 'float32',
 'prep': 'float32',
 'friends': 'float32',
 'present': 'float32',
 'img-label': 'float32',
 'social': 'float32',
 'page_name_adjusted': 'float32',
 'auxvb': 'float32',
 'insight': 'float32',
 'excl': 'float32',
 'percent_dollars': 'float32',
 'percent_questionms': 'float32',
 'sentiment_neu': 'float32',
 'negemo': 'float32',
 'article_subtitle': 'float32',
 'img_src': 'float32',
 'ingest': 'float32',
 'num_links': 'float32',
 'relig': 'float32',
 'feel': 'float32',
 'we': 'float32',
 'quant': 'float32',
 'text_tokenized_filtered': 'float32',
 'conj': 'float32',
 'leisure': 'float32',
 'sc': 'float32',
 'links': 'float32',
 'home': 'float32',
 'text_length': 'float32',
 'future': 'float32',
 'sixltr': 'float32',
 'num_hashtags': '

In [ ]:
posts.drop_duplicates()